# WosClient

In [1]:
# !pip install suds-jurko

from suds import client
from base64 import b64encode as _b64encode
from collections import OrderedDict as _OrderedDict

class WosClient():
    """Query the Web of Science.
       You must provide user and password only to user premium WWS service.
       with WosClient() as wos:
           results = wos.search(...)"""

    base_url = 'http://search.webofknowledge.com'
    auth_url = base_url + '/esti/wokmws/ws/WOKMWSAuthenticate?wsdl'
    search_url = base_url + '/esti/wokmws/ws/WokSearch?wsdl'
    searchlite_url = base_url + '/esti/wokmws/ws/WokSearchLite?wsdl'

    def __init__(self, user=None, password=None, SID=None, close_on_exit=True,
                 lite=False):
        """Create the SOAP clients. user and password for premium access."""

        self._SID = SID
        self._close_on_exit = close_on_exit
        search_wsdl = self.searchlite_url if lite else self.search_url
        self._auth = client.Client(self.auth_url)
        self._search = client.Client(search_wsdl)

        if user and password:
            auth = '%s:%s' % (user, password)
            auth = _b64encode(auth.encode('utf-8')).decode('utf-8')
            headers = {'Authorization': ('Basic %s' % auth).strip()}
            self._auth.set_options(headers=headers)

    def __enter__(self):
        """Automatically connect when used with 'with' statements."""
        self.connect()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """Close connection after closing the 'with' statement."""
        if self._close_on_exit:
            self.close()

    def __del__(self):
        """Close connection when deleting the object."""
        if self._close_on_exit:
            self.close()

    def connect(self):
        """Authenticate to WOS and set the SID cookie."""
        if not self._SID:
            self._SID = self._auth.service.authenticate()
            print('Authenticated (SID: %s)' % self._SID)

        self._search.set_options(headers={'Cookie': 'SID="%s"' % self._SID})
        self._auth.options.headers.update({'Cookie': 'SID="%s"' % self._SID})
        return self._SID

    def close(self):
        """Close the session."""
        if self._SID:
            self._auth.service.closeSession()
            self._SID = None

    def search(self, query, count=5, offset=1):
    # def search(self, query, offset=1):
        """Perform a query. Check the WOS documentation for v3 syntax."""
        if not self._SID:
            raise RuntimeError('Session not open. Invoke .connect() before.')

        qparams = _OrderedDict([('databaseId', 'WOS'),
                                ('userQuery', query),
                                ('queryLanguage', 'en')])

        rparams = _OrderedDict([('firstRecord', offset),
                                ('count', count),
                                ('sortField', _OrderedDict([('name', 'RS'),
                                                            ('sort', 'D')]))])

        return self._search.service.search(qparams, rparams)

In [2]:

from xml.etree import ElementTree as _ET
from xml.dom import minidom as _minidom
import re as _re

def single(wosclient, wos_query, xml_query=None, count=10, offset=1):
    """Perform a single Web of Science query and then XML query the results."""
    result = wosclient.search(wos_query, count, offset)
    print (result)
    xml = _re.sub(' xmlns="[^"]+"', '', result.records, count=1).encode('utf-8')
    if xml_query:
        xml = _ET.fromstring(xml)
        return [el.text for el in xml.findall(xml_query)]
    else:
        return _minidom.parseString(xml).toprettyxml()

def query(wosclient, wos_query, xml_query=None, count=10, offset=1, limit=100):
    """Query Web of Science and XML query results with multiple requests."""
    results = [single(wosclient, wos_query, xml_query, min(limit, count-x+1), x) for x in range(offset, count+1, limit)]
    if xml_query:
        return [el for res in results for el in res]
    else:
        pattern = _re.compile(r'.*?<records>|</records>.*', _re.DOTALL)
        return ('<?xml version="1.0" ?>\n<records>' +
                '\n'.join(pattern.sub('', res) for res in results) +
                '</records>')

def doi_to_wos(wosclient, doi):
    """Convert DOI to WOS identifier."""
    results = query(wosclient, 'DO=%s' % doi, './REC/UID', count=1)
    return results[0].lstrip('WOS:') if results else None

In [3]:
soap = WosClient(lite=True)
soap.connect()

Authenticated (SID: D5YArrhYG5iKQ6d1jHa)


D5YArrhYG5iKQ6d1jHa

# Exchange the Given and Family Name

In [8]:
searchnames = []
i=0
for line in open('Dismis_Acad_List_CLEAN.txt','r'):
    QueryString = line.strip()
    QueryString = "AU="+line.split(',')[0].split()[-1]+' '+line.split(',')[0].split()[0]
    print('QueryString:', QueryString)
    searchnames.append(QueryString)
set(searchnames)

QueryString: AU=AYDIN AYDIN
QueryString: AU=GILGIL ERDAL
QueryString: AU=HALAC METIN
QueryString: AU=YUCE ABDULHAKIM
QueryString: AU=KIRIS ABDULKADIR
QueryString: AU=KUCUKBAYRAK ABDULKADIR
QueryString: AU=YILDIRIM ABDULKADIR
QueryString: AU=CEKIN ABDULKADIR
QueryString: AU=SENGUN ABDULKADIR
QueryString: AU=BEDIR ABDULKERIM
QueryString: AU=AKPINAR ABDULLAH
QueryString: AU=BAL ABDULLAH
QueryString: AU=SENGUL ABDULLAH
QueryString: AU=DEMIR ABDULLAH
QueryString: AU=DOGAN ABDULLAH
QueryString: AU=EKIZER ABDULLAH
QueryString: AU=ERDIL ABDULLAH
QueryString: AU=ERGIN ABDULLAH
QueryString: AU=SECKIN ABDULLAH
QueryString: AU=KIRAN ABDULLAH
QueryString: AU=USTUN ABDULLAH
QueryString: AU=CINGU ABDULLAH
QueryString: AU=AKSOY ABDULLAH
QueryString: AU=TUNCER ABDULLAH
QueryString: AU=MUTLU ABDULLAH
QueryString: AU=KILDAN ABDULLAH
QueryString: AU=POS ABDULLAH
QueryString: AU=SONMEZ ABDULLAH
QueryString: AU=URAL ABDULLAH
QueryString: AU=YORDEM ABDULLAH
QueryString: AU=TARIKULU ABDULLAH
QueryString: AU=C

QueryString: AU=HALAT ERDOGAN
QueryString: AU=KILICASLAN ERDOGAN
QueryString: AU=KOCAMAZ ERDOGAN
QueryString: AU=OZBAY ERDOGAN
QueryString: AU=OZEL ERDOGAN
QueryString: AU=OZKAYA ERDOGAN
QueryString: AU=SENTURK ERDOGAN
QueryString: AU=AKYILDIRIM ERDINC
QueryString: AU=AKSU EREN
QueryString: AU=YILDIRIM EREN
QueryString: AU=GONCA ERGUN
QueryString: AU=GUNTEKIN ERGUN
QueryString: AU=KARAVELIOGLU ERGUN
QueryString: AU=NART ERGUN
QueryString: AU=YASAR ERGUN
QueryString: AU=SAKALAR ERGUN
QueryString: AU=BINGOLBALI ERHAN
QueryString: AU=DELEN ERHAN
QueryString: AU=GENC ERHAN
QueryString: AU=GUNEYISI ERHAN
QueryString: AU=KELESOGLU ERHAN
QueryString: AU=METIN ERHAN
QueryString: AU=SARI ERHAN
QueryString: AU=TAHAN ERHAN
QueryString: AU=VAROGLU ERHAN
QueryString: AU=OZCAN ERHAN
QueryString: AU=UNLU ERHAN
QueryString: AU=KOMURCU ERKAM
QueryString: AU=CURE ERKAN
QueryString: AU=EFE ERKAN
QueryString: AU=GOCEN ERKAN
QueryString: AU=KAVAS ERKAN
QueryString: AU=KACAN ERKAN
QueryString: AU=KURT ERKAN

QueryString: AU=DASCI MAHMUT
QueryString: AU=DEMIRTAS MAHMUT
QueryString: AU=DUYMUS MAHMUT
QueryString: AU=GUZEL MAHMUT
QueryString: AU=KOCAK MAHMUT
QueryString: AU=NALBANTCILAR MAHMUT
QueryString: AU=ULUSOY MAHMUT
QueryString: AU=YALCIN MAHMUT
QueryString: AU=YENER MAHMUT
QueryString: AU=ZKAN MAHMUT
QueryString: AU=OZDEVECIOGLU MAHMUT
QueryString: AU=ULESIN MAHMUT
QueryString: AU=INANC MAHMUT
QueryString: AU=SEKER MAHMUT
QueryString: AU=ALKAN MAHIR
QueryString: AU=ERKURAN MANSUR
QueryString: AU=NART MARUF
QueryString: AU=GOK MAIDE
QueryString: AU=AYHAN MEDIHA
QueryString: AU=CURE MEDINE
QueryString: AU=KARA MEDINE
QueryString: AU=PAKDEMIRLI MEFKURE
QueryString: AU=YEKSAN MEHDI
QueryString: AU=AKBULAK MEHMET
QueryString: AU=AKBULUT MEHMET
QueryString: AU=AKIN MEHMET
QueryString: AU=AKKOSE MEHMET
QueryString: AU=AKSOY MEHMET
QueryString: AU=AKTAS MEHMET
QueryString: AU=CEVIZ MEHMET
QueryString: AU=DOKUZOGLU MEHMET
QueryString: AU=KILAVUZ MEHMET
QueryString: AU=KUTUK MEHMET
QueryString: 

QueryString: AU=DEMIRDAG OKTAY
QueryString: AU=OZTURK OKTAY
QueryString: AU=HATIPOGLU OKTAY
QueryString: AU=SARI OKTAY
QueryString: AU=OKSUZLER OKTAY
QueryString: AU=ESER OLCAY
QueryString: AU=HISAR OLCAY
QueryString: AU=KARAATLI OLCAY
QueryString: AU=KURSUN OLCAY
QueryString: AU=SINAN OLCAY
QueryString: AU=HAMZAOGLU ONUR
QueryString: AU=NANE ONUR
QueryString: AU=OKTAY ONUR
QueryString: AU=SOKUCU ORAL
QueryString: AU=ADIGUZEL ORHAN
QueryString: AU=AKMAN ORHAN
QueryString: AU=ATES ORHAN
QueryString: AU=BOZOGLAN ORHAN
QueryString: AU=DOGDU ORHAN
QueryString: AU=ER ORHAN
QueryString: AU=FILIZ ORHAN
QueryString: AU=GOZENELI ORHAN
QueryString: AU=GUNAY ORHAN
QueryString: AU=KARATAS ORHAN
QueryString: AU=KARPUZ ORHAN
QueryString: AU=KOCAK ORHAN
QueryString: AU=SARIKAYA ORHAN
QueryString: AU=TEZCAN ORHAN
QueryString: AU=TOKGOZ ORHAN
QueryString: AU=OZKAN ORHAN
QueryString: AU=CELTIKCI ORHAN
QueryString: AU=OZBEK ORHAN
QueryString: AU=UNAL ORHAN
QueryString: AU=ZORBA ORHAN
QueryString: AU=ALKI

QueryString: AU=DEMIR IBRAHIM
QueryString: AU=DORUK IBRAHIM
QueryString: AU=ERBAGCI IBRAHIM
QueryString: AU=ERCAN IBRAHIM
QueryString: AU=ERDOGAN IBRAHIM
QueryString: AU=EREN IBRAHIM
QueryString: AU=KARABABA IBRAHIM
QueryString: AU=HEPSEN IBRAHIM
QueryString: AU=GELINCIK IBRAHIM
QueryString: AU=GULER IBRAHIM
QueryString: AU=GUNGOR IBRAHIM
QueryString: AU=HABACI IBRAHIM
QueryString: AU=OZTURK IBRAHIM
QueryString: AU=TASDEMIR IBRAHIM
QueryString: AU=KAYA IBRAHIM
QueryString: AU=KELES IBRAHIM
QueryString: AU=KIR IBRAHIM
QueryString: AU=ONAL IBRAHIM
QueryString: AU=KOSE IBRAHIM
QueryString: AU=OCAK IBRAHIM
QueryString: AU=SARI IBRAHIM
QueryString: AU=ARDIC IBRAHIM
QueryString: AU=SONMEZ IBRAHIM
QueryString: AU=TEGIN IBRAHIM
QueryString: AU=TOBI IBRAHIM
QueryString: AU=TURKYILMAZ IBRAHIM
QueryString: AU=UCAR IBRAHIM
QueryString: AU=YAKUT IBRAHIM
QueryString: AU=USLU IBRAHIM
QueryString: AU=CETINER IBRAHIM
QueryString: AU=OZBEK IBRAHIM
QueryString: AU=KABOGLU IBRAHIM
QueryString: AU=OZDOGRU 

{'AU=ESKIKURT HALIL',
 'AU=TASLI MEHMET',
 'AU=CEYLAN KADIR',
 'AU=TELLIOGLU AYDA',
 'AU=DIKMEN BAYAZIT',
 'AU=EGE ILHAMI',
 'AU=KACIRE ILHAN',
 'AU=SEN GUL',
 'AU=UZUM HAMZA',
 'AU=CINKAYA MUHAMMED',
 'AU=AYVAZ ZAFER',
 'AU=UNAL MUSTAFA',
 'AU=UNAL BIRSEN',
 'AU=ERDEM IDRIS',
 'AU=ALAN BIRCAN',
 'AU=EKEN CENKER',
 'AU=PALABIYIK HAMIT',
 'AU=SELCUK ENGIN',
 'AU=BODUR ABDURRAHMAN',
 'AU=SOGUT ERKAN',
 'AU=BILGE SEMIH',
 'AU=CITISLI VELI',
 'AU=AYNALI GIRAY',
 'AU=BAYRAM OMER',
 'AU=BALKAN AYSE',
 'AU=UYANIK EMRAH',
 'AU=TANYERI BUKET',
 'AU=DOGDU ORHAN',
 'AU=DOGAN BEKIR',
 'AU=CELIK AHMET',
 'AU=AKSU MUHAMMET',
 'AU=CAGLAYA KASIM',
 'AU=TUZCU NEVIN',
 'AU=AKBULAK MEHMET',
 'AU=DEMIR SULEYMAN',
 'AU=KOKSAL MUSTAFA',
 'AU=KOKSAL BAYRAM',
 'AU=OZHAN HAKAN',
 'AU=ALTUNTOP BESIR',
 'AU=KENTLI FULYA',
 'AU=YILMAZ SAZIYE',
 'AU=DURMAN MUSTAFA',
 'AU=BALCI SERDAR',
 'AU=SEN BILGE',
 'AU=AKCAKOCA HAMDI',
 'AU=MUFTUOGLU ALI',
 'AU=SAHIN SALIM',
 'AU=ATA NURCAN',
 'AU=TEMIZ BILAL',
 'AU=HAZAR MEH

In [6]:
len(searchnames.unique())

AttributeError: 'list' object has no attribute 'unique'

In [11]:
import pandas as pd
import time as time

In [7]:
pd.DataFrame(searchnames,columns=['SearchNames']).to_csv('SearchNames.csv')

In [8]:
SearchNames = pd.read_csv('SearchNames.csv')

In [9]:
SearchNames.SearchNames[0].strip().split('=')[1]

'AYDIN AYDIN'

In [12]:
names = []
lens = []
i=0
for i in range(len(SearchNames)):
    line = SearchNames.SearchNames[i]
    QueryString = line.strip()
    print(QueryString)
    if i == 2400:
        soap = WosClient(lite=True)
        soap.connect()
    results = soap.search(QueryString)
    print(results.recordsFound)
    names.append(QueryString.split('=')[1])
    lens.append(results.recordsFound)
    time.sleep(0.5)
    i=i+1

AU=AYDIN AYDIN
4
AU=GILGIL ERDAL
5
AU=HALAC METIN
63
AU=YUCE ABDULHAKIM
0
AU=KIRIS ABDULKADIR
54
AU=KUCUKBAYRAK ABDULKADIR
30
AU=YILDIRIM ABDULKADIR
33
AU=CEKIN ABDULKADIR
0
AU=SENGUN ABDULKADIR
13
AU=BEDIR ABDULKERIM
38
AU=AKPINAR ABDULLAH
38
AU=BAL ABDULLAH
26
AU=SENGUL ABDULLAH
3
AU=DEMIR ABDULLAH
54
AU=DOGAN ABDULLAH
112
AU=EKIZER ABDULLAH
13
AU=ERDIL ABDULLAH
5
AU=ERGIN ABDULLAH
3
AU=SECKIN ABDULLAH
1
AU=KIRAN ABDULLAH
0
AU=USTUN ABDULLAH
5
AU=CINGU ABDULLAH
46
AU=AKSOY ABDULLAH
1
AU=TUNCER ABDULLAH
0
AU=MUTLU ABDULLAH
0
AU=KILDAN ABDULLAH
2
AU=POS ABDULLAH
0
AU=SONMEZ ABDULLAH
22
AU=URAL ABDULLAH
1
AU=YORDEM ABDULLAH
0
AU=TARIKULU ABDULLAH
0
AU=CIM ABDULLAH
14
AU=INALCIK ABDULLAH
5
AU=ASLAN ABDULSELAM
8
AU=GOK ABDULVAHAP
10
AU=OZTURK ABDURAHMAN
0
AU=ABAKAY ABDURRAHMAN
34
AU=DALGIC ABDURRAHMAN
0
AU=ELMALI ABDURRAHMAN
1
AU=GENC ABDURRAHMAN
21
AU=KADAYIFCI ABDURRAHMAN
53
AU=KUZHAN ABDURRAHMAN
4
AU=KIREMITCI ABDURRAHMAN
20
AU=BODUR ABDURRAHMAN
4
AU=SAHIN ABDURRAHMAN
64
AU=ELDEMIR ABD

35
AU=KORKMAZ AYSE
58
AU=SAGIROGLU AYSE
9
AU=SEZGIN AYSE
7
AU=KURT AYSE
12
AU=ULU AYSE
1
AU=YIGIT AYSE
19
AU=KIZILKAN AYSE
0
AU=SARAC AYSEGUL
14
AU=KALKAN AYSEGUL
0
AU=TURKYILMAZ AYSEGUL
30
AU=KURNAZ AYSEGUL
0
AU=AYSEUMUL AYSEUMUL
0
AU=ULUDAG AYSEUL
0
AU=ILBAY AZMI
1
AU=ILHAN AZIME
1
AU=ASAN AZIZ
0
AU=GUMUS AZIZ
17
AU=HARMAN AZIZ
6
AU=KILINC AZIZ
2
AU=MERHAN AZIZ
2
AU=POLAT AZIZ
31
AU=TUNC AZIZ
3
AU=TURKOGLU AZIZ
19
AU=ARTIS AISE
1
AU=GENC ASIR
10
AU=ORENC ASIR
0
AU=YILDIRIM ASKIN
0
AU=SEKER ASKIN
43
AU=OZDEMIR ASUR
0
AU=GUNEY BAHA
0
AU=BOZOGLAN BAHADIR
6
AU=KUMRAL BAHADIR
16
AU=TANTAY BAHADIR
0
AU=ERDINC BAHATTIN
13
AU=TUNC BAHATTIN
88
AU=ERMIS BAHRI
15
AU=ARSLAN BAHRIYE
5
AU=DURSUN BAHTIYAR
19
AU=SARIBOGA BAHTIYAR
13
AU=SARITAS BAKI
0
AU=TURKKAN BAKI
6
AU=ALTAN BANU
0
AU=KALE BANU
4
AU=UYSAL BANU
12
AU=IZGI BANU
0
AU=GUNAY BANIHAN
12
AU=COREKCI BARAM
0
AU=GENCER BARAN
37
AU=ASCI BARIS
3
AU=ERCAL BARIS
17
AU=GULHAN BARIS
12
AU=KIREMITCI BARIS
2
AU=YALINKILIC BARIS
5
AU=ALKAWASMEI BASS

AU=SAKALAR ERGUN
13
AU=BINGOLBALI ERHAN
11
AU=DELEN ERHAN
11
AU=GENC ERHAN
24
AU=GUNEYISI ERHAN
66
AU=KELESOGLU ERHAN
0
AU=METIN ERHAN
0
AU=SARI ERHAN
28
AU=TAHAN ERHAN
4
AU=VAROGLU ERHAN
29
AU=OZCAN ERHAN
14
AU=UNLU ERHAN
23
AU=KOMURCU ERKAM
30
AU=CURE ERKAN
51
AU=EFE ERKAN
13
AU=GOCEN ERKAN
4
AU=KAVAS ERKAN
0
AU=KACAN ERKAN
6
AU=KURT ERKAN
10
AU=SOGUT ERKAN
29
AU=YAMAN ERKAN
5
AU=YULA ERKAN
31
AU=CALISKAN ERKAN
10
AU=SIMSEK ERKAN
2
AU=BAGCIOGLU ERMAN
23
AU=GENC ERNUR
0
AU=ALBAYRAK EROL
5
AU=ERCAG EROL
25
AU=GUNDUZ EROL
1
AU=TOKSEN EROL
5
AU=CELIK EROL
0
AU=COSKUN EROL
25
AU=ONEN EROL
4
AU=CICEK ERSAN
14
AU=DAG ERSEL
27
AU=OBUZ ERSEL
16
AU=ERKAN ERSIN
0
AU=FADILLIOGLU ERSIN
32
AU=KARAPAZARLIOGLU ERSIN
0
AU=TERES ERSIN
1
AU=ERDEN ERSIN
9
AU=BESE ERTAN
0
AU=DURMUSOGLU ERTAN
16
AU=ERSOY ERTAN
0
AU=YANIK ERTAN
9
AU=OZTURK ERTAN
20
AU=ACARTURK ERTUGRUL
0
AU=ERCAN ERTUGRUL
49
AU=KARGI ERTUGRUL
15
AU=MAVILI ERTUGRUL
43
AU=UZUN ERTUGRUL
3
AU=BOZKURT ESAT
18
AU=CETIN ESAT
3
AU=SAVAS ESEN
24
AU

AU=BABACAN HASAN
22
AU=BAL HASAN
14
AU=ERTAS HASAN
25
AU=ULUSOY HASAN
40
AU=BAYRAKTAR HASAN
3
AU=BOZKURT HASAN
30
AU=BUKER HASAN
5
AU=BUYUKASLAN HASAN
14
AU=BILINC HASAN
13
AU=EKER HASAN
31
AU=ELLEK HASAN
0
AU=ERDOGAN HASAN
50
AU=ERDOGAN HASAN
50
AU=ERTAS HASAN
25
AU=ERCELIK HASAN
0
AU=EROL HASAN
2
AU=GOK HASAN
35
AU=GUNGOR HASAN
59
AU=ULUTIN HASAN
0
AU=DULGER HASAN
1
AU=HERKEN HASAN
65
AU=KART HASAN
3
AU=KOSGER HASAN
0
AU=SATILMIS HASAN
6
AU=YURTCU HASAN
1
AU=CALI HASAN
0
AU=CEVIK HASAN
3
AU=OZDEMIR HASAN
82
AU=YUCE HASAN
23
AU=KAFALI HASAN
52
AU=KALA HASAN
0
AU=KAMAK HASAN
11
AU=KAPLAN HASAN
26
AU=KARACAN HASAN
0
AU=KARAKURT HASAN
3
AU=KAYA HASAN
155
AU=KURT HASAN
15
AU=METINEREN HASAN
11
AU=MUTLU HASAN
83
AU=OZER HASAN
59
AU=SARAL HASAN
7
AU=GERGERLIOGLU HASAN
15
AU=TAK HASAN
1
AU=ATMACA HASAN
26
AU=TURKMEN HASAN
14
AU=UCMAK HASAN
18
AU=YILDI HASAN
0
AU=YILDIZHAN HASAN
6
AU=YUMAK HASAN
8
AU=CANKAYA HASAN
0
AU=CEVIKOGLU HASAN
0
AU=CECE HASAN
47
AU=CINAR HASAN
1
AU=OZTURK HASAN
74
AU=

AU=KOCOGLU KIYASETTIN


WebFault: Server raised fault: 'Request denied by Throttle server. Request denied for quantity of 1 for activity "Requests" of service "WOK-WS" for id "e06b8e16e0215870afc8befa74e7458e.27994" on Wednesday, November 21, 2018 at 1:35:35 PM UTC.  The following reasons apply:   Reason (1) - limit of 2 requests per period has been exceeded for throttle REQ-ReqPerSec. Details of this throttle are the following : this is a RequestPerPeriod throttle; the period length is 1 second(s); the period restarts after Wednesday, November 21, 2018 at 1:35:35 PM UTC.'

In [68]:
df = pd.DataFrame([names,lens]).T
df.columns=['Author','NoPapers']

In [71]:
# 753 out of 3575 people have 0 papers in the web of science dataset
zeropapers = df.loc[df.NoPapers==0].copy()

In [72]:
# df.to_csv('Publishments.csv')

In [73]:
zeropapers.to_csv('ZeroPublishment.csv')

In [84]:
pd.concat([SearchNames,df],axis=1).to_csv('Publishments.csv')

In [85]:
pd.read_csv('Publishments.csv')

,Unnamed: 0,SearchNames,Author,NoPapers
0,0,AU=AYDIN AYDIN,AYDIN AYDIN,4
1,1,AU=GILGIL ERDAL,GILGIL ERDAL,5
2,2,AU=HALAC METIN,HALAC METIN,63
3,3,AU=YUCE ABDULHAKIM,YUCE ABDULHAKIM,0
4,4,AU=KIRIS ABDULKADIR,KIRIS ABDULKADIR,54
5,5,AU=KUCUKBAYRAK ABDULKADIR,KUCUKBAYRAK ABDULKADIR,30
6,6,AU=YILDIRIM ABDULKADIR,YILDIRIM ABDULKADIR,33
7,7,AU=CEKIN ABDULKADIR,CEKIN ABDULKADIR,0
8,8,AU=SENGUN ABDULKADIR,SENGUN ABDULKADIR,13
9,9,AU=BEDIR ABDULKERIM,BEDIR ABDULKERIM,38
